##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 데이터 검증

***TensorFlow Extended의 주요 구성 요소 예***

참고: 이 예제는 Jupyter 스타일 노트북에서 바로 실행할 수 있으며 설정이 필요하지 않습니다. "Google Colab에서 실행"을 클릭하세요.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/data_validation/tfdv_basic.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/data_validation/tfdv_basic.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tfx/tutorials/data_validation/tfdv_basic.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table></div>

이 예제 colab 노트북은 TensorFlow 데이터 유효성 검사(TFDV)를 사용하여 데이터세트를 조사하고 시각화하는 방법을 보여줍니다. 여기에는 기술 통계 보기, 스키마 추론, 이상 항목 확인 및 수정, 데이터세트의 드리프트 및 왜곡 확인이 포함됩니다. 프로덕션 파이프라인에서 시간이 지남에 따라 변경될 수 있는 방식을 포함하여 데이터세트의 특성을 이해하는 것이 중요합니다. 또한 데이터에서 이상을 찾고 훈련, 평가 및 제공 데이터세트를 비교하여 일관성이 있는지 확인하는 것도 중요합니다.

시카고 시에서 발표한 [Taxi Trips 데이터세트](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)의 데이터를 사용합니다.

참고: 이 사이트는 원 출처인 시카고 시의 공식 웹 사이트 www.cityofchicago.org를 바탕으로 수정된 데이터를 사용하는 애플리케이션을 제공합니다. 시카고 시는 이 사이트에서 제공되는 데이터의 내용, 정확성, 적시성 또는 완전성에 대해 어떠한 주장도하지 않습니다. 이 사이트에서 제공되는 데이터는 언제든지 변경될 수 있습니다. 이 사이트에서 제공하는 데이터는 자신의 책임 하에 사용되는 것으로 이해됩니다.

[Google BigQuery](https://cloud.google.com/bigquery/public-data/chicago-taxi)의 데이터세트에 대해 [자세히 알아보세요](https://cloud.google.com/bigquery/). [BigQuery UI](https://bigquery.cloud.google.com/dataset/bigquery-public-data:chicago_taxi_trips)에서 전체 데이터세트를 살펴보세요.

핵심 포인트: 모델러 및 개발자로서 이 데이터가 어떻게 사용되는지, 그리고 모델의 예측이 초래할 수 있는 잠재적인 이점과 피해에 대해 생각해보세요. 이와 같은 모델은 사회적 편견과 불균형을 강화시킬 수 있습니다. 기능이 해결하려는 문제와 관련이 있습니까? 아니면 편견을 유발합니까? 자세한 내용은 [ML 공정성](https://developers.google.com/machine-learning/fairness-overview/)에 대해 읽어보세요.

데이터세트의 열은 다음과 같습니다.

<table>
<tr>
<td>pickup_community_area</td>
<td>fare</td>
<td>trip_start_month</td>
</tr>
<tr>
<td>trip_start_hour</td>
<td>trip_start_day</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>pickup_latitude</td>
<td>pickup_longitude</td>
<td>dropoff_latitude</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>trip_miles</td>
<td>Pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>payment_type</td>
<td>company</td>
</tr>
<tr>
<td>trip_seconds</td>
<td>dropoff_community_area</td>
<td>tips</td>
</tr>
</table>

## 패키지 설치 및 가져오기

TensorFlow 데이터 유효성 검사용 패키지를 설치합니다.

### Pip 업그레이드

로컬에서 실행할 때 시스템에서 Pip을 업그레이드하지 않으려면 Colab에서 실행 중인지 확인하세요. 물론 로컬 시스템은 별도로 업그레이드할 수 있습니다.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### 데이터 유효성 검사 패키지 설치

TensorFlow 데이터 유효성 검사 패키지 및 종속 항목을 설치합니다. 이 작업은 몇 분 정도 걸립니다. 호환되지 않는 종속성 버전에 대한 경고 및 오류가 표시될 수 있고, 이 문제는 다음 섹션에서 해결할 수 있습니다.

In [ ]:
print('Installing TensorFlow Data Validation')
!pip install --upgrade 'tensorflow_data_validation[visualization]<2'

### TensorFlow 가져오기 및 업데이트된 패키지 다시 로드하기

이전 단계에서 Gooogle Colab 환경의 기본 패키지를 업데이트하므로 패키지 리소스를 다시 로드하여 새 종속성을 해결해야 합니다.

참고: 이 단계는 설치의 종속성 오류를 해결합니다. 이 코드를 실행한 후에도 여전히 코드 실행 문제가 발생하면 런타임을 다시 시작하세요(런타임 &gt; 런타임 다시 시작...).

In [ ]:
import pkg_resources
import importlib
importlib.reload(pkg_resources)

계속하기 전에 TensorFlow와 데이터 유효성 검사 버전을 확인하세요. 

In [ ]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
print('TF version:', tf.__version__)
print('TFDV version:', tfdv.version.__version__)

## 데이터세트 로드하기

Google Cloud Storage에서 데이터세트를 다운로드합니다.

In [ ]:
import os
import tempfile, urllib, zipfile

# Set up some globals for our file paths
BASE_DIR = tempfile.mkdtemp()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'chicago_taxi_output')
TRAIN_DATA = os.path.join(DATA_DIR, 'train', 'data.csv')
EVAL_DATA = os.path.join(DATA_DIR, 'eval', 'data.csv')
SERVING_DATA = os.path.join(DATA_DIR, 'serving', 'data.csv')

# Download the zip file from GCP and unzip it
zip, headers = urllib.request.urlretrieve('https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/chicago_data.zip')
zipfile.ZipFile(zip).extractall(BASE_DIR)
zipfile.ZipFile(zip).close()

print("Here's what we downloaded:")
!ls -R {os.path.join(BASE_DIR, 'data')}

## 통계 계산 및 시각화하기

먼저 [`tfdv.generate_statistics_from_csv`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/generate_statistics_from_csv)를 사용하여 훈련 데이터에 대한 통계를 계산합니다(경고 무시).

TFDV는 존재하는 특징과 그 값 분포의 형태 측면에서 데이터의 빠른 개요를 제공하는 기술 [통계](https://github.com/tensorflow/metadata/blob/v0.6.0/tensorflow_metadata/proto/v0/statistics.proto)를 계산할 수 있습니다.

내부적으로, TFDV는 [Apache Beam](https://beam.apache.org/)의 데이터 병렬 처리 프레임워크를 사용하여 대규모 데이터세트에 대한 통계 계산을 확장합니다. TFDV와 더 심층적으로 통합하려는 애플리케이션의 경우(예: 데이터 생성 파이프라인 끝에 통계 생성 연결) API는 통계 생성을 위해 Beam PTransform도 노출합니다.

In [ ]:
train_stats = tfdv.generate_statistics_from_csv(data_location=TRAIN_DATA)

이제 [Facets](https://pair-code.github.io/facets/)를 사용하여 훈련 데이터의 간결한 시각화를 만드는 [`tfdv.visualize_statistics`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/visualize_statistics)를 사용하겠습니다.

- 숫자 특성과 범주 특성은 별도로 시각화되며 각 특성에 대한 분포를 보여주는 차트가 표시됩니다.
- 값이 누락되거나 0인 특성은 해당 특성의 예제에 문제가 있을 수 있음을 나타내는 시각적 표시로 빨간색으로 표시됩니다. 백분율은 해당 특성에 대한 값이 누락되었거나 0인 예제의 백분율입니다.
- `pickup_census_tract` 값이 있는 예는 없습니다. 차원을 줄일 기회입니다!
- 표시를 변경하려면 차트 위의 "확장"을 클릭하세요.
- 차트의 막대 위로 마우스를 가져가면 버킷 범위와 개수를 표시할 수 있습니다.
- 로그 스케일과 선형 스케일 사이를 전환해 로그 스케일에서 `payment_type` 범주 특성에 대해 훨씬 더 많은 정보가 드러날 수 있음을 확인하세요.
- "표시할 차트" 메뉴에서 "분위수"를 선택하고 마커 위로 마우스를 가져가 분위수 백분율을 표시합니다.

In [ ]:
# docs-infra: no-execute
tfdv.visualize_statistics(train_stats)

<!-- <img class="tfo-display-only-on-site" src="images/statistics.png"/> -->

## 스키마 추론하기

이제 [`tfdv.infer_schema`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/infer_schema)를 사용하여 데이터에 대한 스키마를 생성해 보겠습니다. 스키마는 ML과 관련된 데이터에 대한 제약 조건을 정의합니다. 제약 조건의 예에는 각 특성의 데이터 유형(숫자형이든 범주형이든 상관없이) 또는 데이터에 존재하는 빈도가 포함됩니다. 범주형 특성의 경우 스키마는 허용되는 값 목록인 도메인도 정의합니다. 스키마 작성은 특히 많은 특성이 있는 데이터세트의 경우 지루한 작업이 될 수 있으므로 TFDV는 기술 통계를 기반으로 스키마의 초기 버전을 생성하는 메서드를 제공합니다.

나머지 프로덕션 파이프라인은 TFDV가 생성하는 스키마에 의존하여 문제를 방지하므로 스키마를 올바르게 가져오는 것이 중요합니다. 스키마는 데이터에 대한 문서도 제공하므로 여러 개발자가 동일한 데이터로 작업할 때 유용합니다. [`tfdv.display_schema`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/display_schema)를 사용하여 추론된 스키마를 표시하여 검토할 수 있도록 하겠습니다.

In [ ]:
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=schema)

## 평가 데이터에 오류가 있는지 확인하기

지금까지 우리는 훈련 데이터만 살펴보았습니다. 평가 데이터가 동일한 스키마를 사용하는 것을 포함하여 교육 데이터와 일치하는 것이 중요합니다. 또한 평가 중 손실 표면의 범위가 학습 중일 때와 대략적으로 동일하도록 평가 데이터에 학습 데이터와 수치 특성에 대한 값의 범위가 대략 동일한 예가 포함되는 것이 중요합니다. 범주형 특성의 경우도 마찬가지입니다. 그렇지 않으면 손실 표면의 일부를 평가하지 않았기 때문에 평가 중에 확인되지 않은 학습 문제가 있을 수 있습니다.

- 이제 각 특성에 학습 및 평가 데이터세트에 대한 통계가 포함됩니다.
- 이제 차트에 학습 및 평가 데이터세트가 모두 중첩되어 쉽게 비교할 수 있습니다.
- 이제 차트에 로그 또는 기본 선형 배율과 결합할 수 있는 백분율 보기가 포함됩니다.
- `trip_miles`의 평균과 중앙값은 학습 데이터세트와 평가 데이터세트에서 서로 다릅니다. 이것이 문제를 일으킬까요?
- 와우, 최대 `tips`는 학습 데이터세트와 평가 데이터세트 사이에서 서로 매우 다릅니다. 이것이 문제를 일으킬까요?
- 숫자 특성 차트에서 확장을 클릭하고 로그 스케일을 선택합니다. `trip_seconds` 특성을 검토하고 최대값의 차이를 확인합니다. 평가에서 손실 표면의 일부가 빠질까요?

In [ ]:
# Compute stats for evaluation data
eval_stats = tfdv.generate_statistics_from_csv(data_location=EVAL_DATA)

In [ ]:
# docs-infra: no-execute
# Compare evaluation data with training data
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

<!-- <img class="tfo-display-only-on-site" src="images/statistics_eval.png"/> -->

## 평가 이상 여부 확인하기

평가 데이터세트가 학습 데이터세트의 스키마와 일치합니까? 이는 허용 가능한 값의 범위를 식별하려는 범주형 특성에 특히 중요합니다.

핵심 포인트: 학습 데이터세트에 없는 범주형 특성 값을 가진 데이터를 사용하여 평가하려고 하면 어떻게 될까요? 학습 데이터세트의 범위를 벗어난 숫자 특성은 어떻습니까?

In [ ]:
# Check eval data for errors by validating the eval data stats using the previously inferred schema.
anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)
tfdv.display_anomalies(anomalies)

## 스키마의 평가 이상 수정하기

이런! 평가 데이터에 있지만 학습 데이터에는 없었던 `company`에 대한 일부 새로운 값이 있는 것 같습니다. `payment_type`에도 새로운 값이 있습니다. 이러한 값은 비정상적인 것으로 간주되어야 하지만 이에 대해 우리가 내리는 결정은 데이터에 대한 도메인 지식에 달려 있습니다. 이상이 실제로 데이터 오류를 나타내는 경우, 기본 데이터를 수정해야 합니다. 그렇지 않으면 평가 데이터세트에 값을 포함하도록 간단히 스키마를 업데이트할 수 있습니다.

핵심 포인트: 이러한 문제를 해결하지 않으면 평가 결과에 어떤 영향이 있을까요?

평가 데이터세트를 변경하지 않는 한 모든 것을 수정할 수는 없지만 수용하기 편한 스키마 내용을 수정할 수 있습니다. 여기에는 특정 특성에 대해 무엇이 이상이고 무엇이 이상이 아닌지에 대한 우리의 견해를 완화하고 범주형 특성에 대해 누락된 값을 포함하도록 스키마를 업데이트하는 것이 포함됩니다. TFDV를 통해 수정해야 할 내용을 발견할 수 있었습니다.

지금 이러한 수정을 해보고 한 번 더 검토하겠습니다.

In [ ]:
# Relax the minimum fraction of values that must come from the domain for feature company.
company = tfdv.get_feature(schema, 'company')
company.distribution_constraints.min_domain_mass = 0.9

# Add new value to the domain of feature payment_type.
payment_type_domain = tfdv.get_domain(schema, 'payment_type')
payment_type_domain.value.append('Prcard')

# Validate eval stats after updating the schema 
updated_anomalies = tfdv.validate_statistics(eval_stats, schema)
tfdv.display_anomalies(updated_anomalies)

보세요! 학습 및 평가 데이터가 이제 일관성이 있음을 확인했습니다! 감사합니다 TFDV ;)

## 스키마 환경

또한, 이 예제에서 '제공' 데이터세트를 분리했으므로 이것도 확인해야 합니다. 기본적으로 파이프라인의 모든 데이터세트는 동일한 스키마를 사용해야 하지만 종종 예외가 있습니다. 예를 들어. 지도 학습에서는 데이터세트에 레이블을 포함해야 하지만 추론을 위해 모델을 제공할 때는 레이블이 포함되지 않습니다. 어떤 경우에는 약간의 스키마 변형을 도입해야 합니다.

**환경**을 사용하여 이러한 요구 사항을 표현할 수 있습니다. 특히, 스키마의 특성은 `default_environment`, `in_environment` 및 `not_in_environment`를 사용하여 일단의 환경과 연관될 수 있습니다.

예를 들어, 이 데이터세트에서 `tips` 특성은 학습을 위한 레이블로 포함되어 있지만 제공 데이터에는 없습니다. 환경을 지정하지 않으면 이상으로 표시됩니다.

In [ ]:
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

아래 `tips` 특성을 다룰 것입니다. 또한 트립 초에 INT 값이 있는데, 여기서 스키마는 FLOAT를 예상했습니다. 그 차이를 인식하게 함으로써 TFDV는 학습 및 제공을 위해 데이터가 생성되는 방식의 불일치를 발견하는 데 도움을 줍니다. 모델 성능이 저하될 때까지(때로는 비극적으로) 이러한 문제를 인식하지 못하기가 매우 쉽습니다. 이것은 중요한 문제일 수도 있고 아닐 수도 있지만 어떤 경우이던 추가적인 검토가 필요합니다.

이 경우 INT 값을 FLOAT로 안전하게 변환할 수 있으므로 TFDV에 스키마를 사용하여 유형을 추론하도록 지시하려고 합니다. 지금 해보겠습니다.

In [ ]:
options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA, stats_options=options)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

이제 `tips` 특성(즉, 라벨)이 이상('열 삭제')으로 표시됩니다. 물론 제공 데이터에 레이블이 있을 것으로 예상하지 않으므로 TFDV에 이를 무시하도록 지시하겠습니다.

In [ ]:
# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Specify that 'tips' feature is not in SERVING environment.
tfdv.get_feature(schema, 'tips').not_in_environment.append('SERVING')

serving_anomalies_with_env = tfdv.validate_statistics(
    serving_stats, schema, environment='SERVING')

tfdv.display_anomalies(serving_anomalies_with_env)

## 드리프트 및 스큐 확인하기

데이터세트가 스키마에 설정된 기대 사항을 따르는지 확인하는 외에도 TFDV는 드리프트 및 스큐를 감지하는 기능도 제공합니다. TFDV는 스키마에 지정된 드리프트/스큐 비교기를 기반으로 여러 데이터세트의 통계를 비교하여 이 검사를 수행합니다.

### 드리프트

드리프트 감지는 범주형 특성 및 데이터의 연속 범위 사이(즉, 범위 N과 범위 N+1 사이)(예: 훈련 데이터의 서로 다른 날 사이)에서 지원됩니다. [L-infinity 거리](https://en.wikipedia.org/wiki/Chebyshev_distance)로 드리프트를 표현하며, 드리프트가 허용 범위보다 높을 때 경고를 받을 수 있도록 임계값 거리를 설정할 수 있습니다. 올바른 거리 설정은 일반적으로 도메인 지식과 실험이 필요한 반복적인 프로세스입니다.

### 스큐

TFDV는 데이터에서 스키마 편향, 특성 편향 및 분포 편향의 세 가지 다른 유형의 편향을 감지 할 수 있습니다.

#### 스키마 왜곡

스키마 편향은 학습 및 제공 데이터가 동일한 스키마를 따르지 않을 때 발생합니다. 학습 데이터와 제공 데이터는 모두 동일한 스키마를 준수해야합니다. 둘 사이의 예상 편차 (예 : 학습 데이터에만 존재하지만 제공에는없는 라벨 기능)는 스키마의 환경 필드를 통해 지정해야합니다.

#### 기능 왜곡

특성 편향은 모델이 학습하는 특성 값이 제공 시간에 표시되는 특성 값과 다를 때 발생합니다. 예를 들어 다음과 같은 경우에 발생할 수 있습니다.

- 일부 특성 값을 제공하는 데이터 소스는 학습과 제공 시간 사이에 수정됩니다.
- 학습과 제공간에 특성을 생성하는 논리가 다릅니다. 예를 들어, 두 코드 경로 중 하나에 만 일부 변환을 적용하는 경우입니다.

#### 분포 왜곡

분포 왜곡은 학습 데이터 세트의 분포가 제공 데이터 세트의 분포와 크게 다를 때 발생합니다. 분포 왜곡의 주요 원인 중 하나는 다른 코드 또는 다른 데이터 소스를 사용하여 학습 데이터 세트를 생성하는 것입니다. 또 다른 이유는 학습 할 제공 데이터의 대표적이지 않은 하위 샘플을 선택하는 잘못된 샘플링 메커니즘입니다.

In [ ]:
# Add skew comparator for 'payment_type' feature.
payment_type = tfdv.get_feature(schema, 'payment_type')
payment_type.skew_comparator.infinity_norm.threshold = 0.01

# Add drift comparator for 'company' feature.
company=tfdv.get_feature(schema, 'company')
company.drift_comparator.infinity_norm.threshold = 0.001

skew_anomalies = tfdv.validate_statistics(train_stats, schema,
                                          previous_statistics=eval_stats,
                                          serving_statistics=serving_stats)

tfdv.display_anomalies(skew_anomalies)

이 예에서는 약간의 드리프트가 표시되지만 설정 한 임계 값보다 훨씬 낮습니다.

## 스키마 고정

스키마가 검토되고 선별되었으므로 이제 "고정"상태를 반영하기 위해 파일에 저장합니다.

In [ ]:
from tensorflow.python.lib.io import file_io
from google.protobuf import text_format

file_io.recursive_create_dir(OUTPUT_DIR)
schema_file = os.path.join(OUTPUT_DIR, 'schema.pbtxt')
tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

## TFDV를 사용하는 경우

TFDV를 여기에서 한 것처럼 훈련 파이프라인의 시작에만 적용하는 것으로 생각하기 쉽지만 실제로는 많은 용도가 있습니다. 다음은 몇 가지 더 있습니다.

- 추론을 위해 새 데이터를 검증하여 갑자기 잘못된 기능을 받기 시작하지 않았는지 확인
- 추론을위한 새 데이터의 유효성을 검사하여 모델이 의사 결정 표면의 해당 부분에 대해 학습했는지 확인
- 데이터를 변환하고 기능 엔지니어링을 수행한 후 데이터 검증(보통 [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) 사용)을 통해 잘못된 작업이 없는지 확인